# Features acteurs oscar

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
µ
from CinePred.data.featuring import *
from CinePred.data.preprocessing import *
from CinePred.data.importing import *

NameError: name 'μ' is not defined

# Aller on taff

In [ ]:
acteurs_df = pd.read_csv('../raw_data/cat_acteur.csv')


In [ ]:
acteurs_df

In [ ]:
df = preprocess_example(path = "../raw_data/IMDb_movies.csv")

In [ ]:
df = add_success_movies_per_actors(df, path = "../raw_data/cat_acteur.csv")

In [ ]:
df = pd.read_csv('../raw_data/cat_acteur.csv')

In [ ]:
df

## On définit le ratio de chaque films

In [ ]:
acteurs_df['ratio'] = acteurs_df['income']/acteurs_df['budget']

### Détermination des catégories des acteurs

In [ ]:
acteurs_df_cat = acteurs_df.loc[(acteurs_df['ratio']>=5) & (acteurs_df['budget'] >= 100_000_000)]

In [ ]:
len(acteurs_df_cat['acteur_name'].unique())

In [ ]:
acteurs_df_cat[['acteur_name','budget']].groupby(by='acteur_name').sum().sort_values(by="budget", ascending=False).head(159)

### Catégorisation

In [ ]:
top_tier_actor_df = acteurs_df.groupby(by="acteur_name").sum()[['budget']].sort_values("budget", ascending=False)

In [ ]:
top_tier_actor_df.head(50)

In [ ]:
def connupaconnu(X, ratio = 3):
    if (X['budget'] >= 100_000_000) & (X['ratio'] >= 3):
        return 1
    return 0

In [ ]:
acteurs_df.sort_values('year', ascending=True, inplace= True)

In [ ]:
acteurs_df['connu'] = acteurs_df.apply(connupaconnu, axis = 1)

In [ ]:
acteurs_df['nbsucces'] = acteurs_df['connu']

In [ ]:
new_df = acteurs_df

In [ ]:
new_df['nbsuccess'] = acteurs_df.groupby(by  ='acteur_name')['connu'].cumsum(axis = 0)

In [ ]:
new_df = new_df.sort_values('year', ascending=True)

In [ ]:
new_df.drop(columns='nbsucces', inplace = True)

In [ ]:
new_df['connu2'] = new_df['nbsuccess'].apply(lambda x : 1 if x >=1 else 0)

In [ ]:
new_df.drop(columns="connu", inplace = True)

In [ ]:
new_df.rename(columns={"connu2" : "connu"}, inplace=True)

In [ ]:
new_df[new_df['title'].str.contains('Star Trek')]

In [ ]:
new_df.drop(index=new_df[new_df['acteur_name'] == "Leon" ].index)

In [ ]:
liste_fi = new_df[['acteur_name','title','year','nbsuccess','connu']].values.tolist()

In [ ]:
new_df['totalsuccess'] = new_df.groupby(by = 'title').cumsum()['nbsuccess']

In [ ]:
# new_df['totalsuccess2'] = new_df.groupby(by = 'title')['totalsuccess']


# df.groupby(['Mt'], sort=False)['count'].max()

In [ ]:
total_success = pd.DataFrame(new_df.groupby(['title'], sort = False)['totalsuccess'].max())

In [ ]:
total_success.reset_index(inplace=True)

In [ ]:
total_success[total_success['title'].str.contains('Harry')]

In [ ]:
total_success.sort_values('totalsuccess', ascending=False).head(50)

# Chercher l'acteur qu'on veut (optionnel)

In [ ]:
acteurs_df[acteurs_df['title'].str.contains("La maschera di ferro")].sort_values('budget', ascending=False).sort_values('year')

In [ ]:
acteurs_df.drop(columns='nbsucces', inplace = True)

In [ ]:
acteurs_df[acteurs_df['acteur_name'].str.contains("Leonardo DiCaprio")]
acteurs_df['shifted'] = acteurs_df['nbsuccess'] - acteurs_df['connu']

In [ ]:
acteurs_df[['acteur_name','title','shifted']].groupby(by="title")

In [ ]:
acteurs_df

In [ ]:
total_success = pd.DataFrame(acteurs_df.groupby(['title'], sort = False)[['acteur_name','shifted']].max())
total_success.reset_index(inplace = True)

In [ ]:
total_success[total_success['acteur_name'].str.contains('Leonardo DiCaprio')]

In [ ]:
acteurs_df[acteurs_df['acteur_name'].str.contains('Margot Robbie')]

In [ ]:
last_df = pd.DataFrame(acteurs_df.groupby('title').sum()["shifted"])

In [ ]:
last_df.reset_index(inplace = True)

In [ ]:
last_df[last_df['title'].str.contains('Avengers')]

In [ ]:
acteurs_df

In [ ]:
laaast = pd.DataFrame(last_df.merge(acteurs_df, on='title').drop(columns=["connu","nbsuccess","shifted_y","Unnamed: 0"])).drop_duplicates()

In [ ]:
# last_df.to_csv('../raw_data/cat_acteur.csv')

In [ ]:
laaast[laaast['title'].str.contains('Avengers: Endgame')]

In [ ]:
laaast.rename(columns={"shifted_x":'shifted'}, inplace=True)

In [ ]:
# laaast.to_csv('../raw_data/cat_acteur.csv')

In [ ]:
print('----- import Data -----')
df = import_data("../raw_data/IMDb movies.csv")

print('----- keep columns -----')
df = keep_columns(df,
                  column_names=[
                      'imdb_title_id', 'title', 'year', 'date_published',
                      'genre', 'duration', 'country', 'director', 'writer',
                      'production_company', 'actors', 'budget',
                      'worlwide_gross_income'
                  ])

print('----- remove na rows -----')
df = remove_na_rows(df)

print('----- convert budget -----')
df['budget'] = convert_budget_column(df[['budget']])

# print('----- reduce column type -----')
# df['actors'] = reduce_column_type(df[['actors']], nb_max=3)

print('----- convert income column -----')
df['worlwide_gross_income'] = convert_income(df[['worlwide_gross_income']])

print('----- convert to int -----')
df['year'] = convert_to_int(df[['year']])
df['duration'] = convert_to_int(df[['duration']])

print('----- convert to date -----')
df['date_published'] = convert_to_date(df[['date_published']])

print('----- reset index -----')
df = reset_index(df)

# print('----- log transform -----')
# df['worlwide_gross_income'] = log_transformation(
#     df[['worlwide_gross_income']])

# print('----- data_shape -----')

In [ ]:
df

In [ ]:
df = add_success_movies_per_actors(df)

In [ ]:
df

In [ ]:
df[df['actors'].str.contains('Leonardo DiCaprio')]

In [ ]:
df['DirectorsOnes'] = 1

In [ ]:
df['nbsuccessDirectors'] = df.groupby(by  ='director').cumsum()['DirectorsOnes']

In [ ]:
df['nbsuccessProdCompa'] = df.groupby(by  ='production_company').cumsum()['DirectorsOnes']

In [ ]:
df['nbsuccessWriter'] = df.groupby(by  ='writer').cumsum()['DirectorsOnes']

In [ ]:
df.sort_values('nbsuccessWriter', ascending=False)